In [1]:
import asyncio
from configparser import RawConfigParser
import logging 
from pathlib import Path

In [2]:
import httpx
from httpx_caching import CachingClient
from httpx_caching._heuristics import ExpiresAfterHeuristic

In [3]:
import uscensus
from uscensus.asyncio.data.discovery import AsyncDiscoveryInterface, AsyncCensusDataEndpoint
from uscensus.asyncio.util import AsyncSqlAlchemyDataStore

In [4]:
logging.basicConfig(level=logging.INFO)
logging.getLogger('uscensus.asyncio.data').setLevel(logging.DEBUG)

In [5]:
cp = RawConfigParser()
cp.read(Path('~/.census').expanduser())

['/home/nkrishna/.census']

In [6]:
api_key = cp['census']['api_key']

In [7]:
async def main():
    global adi
    adi=None
    client = CachingClient(httpx.AsyncClient(follow_redirects=True),
                           cacheable_status_codes=(200,203,300,301,302,308),
                           heuristic=ExpiresAfterHeuristic(days=7),
                           cache=await AsyncSqlAlchemyDataStore.create('sqlite+aiosqlite:///testing.db'))
    try:
        adi = await AsyncDiscoveryInterface.create(
                cp.get('census','api_key'),
                client, 
                vintage='1986/cbp')
        return adi
    except Exception as e:
        logging.exception(e)
        raise

In [8]:
loop = asyncio.get_event_loop()

In [9]:
task = loop.create_task(main())

DEBUG:uscensus.asyncio.data.discovery:Fetching root metadata
INFO:uscensus.asyncio.data.discovery:Processing 1 datasets
DEBUG:uscensus.asyncio.data.discovery:Processing dataset 1986/cbp
For more information check: https://httpstatuses.com/404
DEBUG:uscensus.asyncio.data.discovery:Finished processing metadata for dataset: 1986/cbp
DEBUG:uscensus.asyncio.data.discovery:Indexing metadata for 1986/cbp
DEBUG:uscensus.asyncio.data.discovery:Indexed metadata for 1986/cbp
INFO:uscensus.asyncio.data.discovery:Done processing datasets


In [10]:
adi = task.result()
adi

{'1986/cbp': 1986 County Business Patterns: Business Patterns}

In [11]:
results = adi.search('"county"')
results

,score,dataset_id,title,description
0,-0.000003,1986/cbp,1986 County Business Patterns: Business Patterns,County Business Patterns (CBP) is an annual se...


In [12]:
api = adi[results.loc[0, 'dataset_id']]
api

1986 County Business Patterns: Business Patterns

In [13]:
api.variables

,label,concept,predicateType,group,limit,predicateOnly,attributes
for,Census API FIPS 'for' clause,Census API Geography Specification,fips-for,N/A,0,True,NaN
in,Census API FIPS 'in' clause,Census API Geography Specification,fips-in,N/A,0,True,NaN
ESTAB_F,Flag for Total number of Establishments,Employer Statistics,int,N/A,0,NaN,NaN
GEO_TTL,Title of Geography,Geographic Characteristics,NaN,N/A,0,NaN,NaN
ST,FIPS State Code,Selectable Geographies,NaN,N/A,0,NaN,NaN
PAYQTR1,Total First Quarter Payroll,Employer Statistics,int,N/A,0,NaN,NaN
EMPSZES_TTL,Title of Employment size of establishment,Employer Statistics,string,N/A,0,NaN,NaN
PAYQTR1_F,Flag for Total First Quarter Payroll,Employer Statistics,string,N/A,0,NaN,NaN
SIC_TTL,Title of SIC code,CBP County Level Variables,NaN,N/A,0,NaN,NaN
EMP,Total Number of Employees,Employer Statistics,int,N/A,0,NaN,NaN


In [14]:
vars = api.searchVariables('employees')
vars

,score,variable,group,label,concept
0,-2.155604,EMP,N/A,Total Number of Employees,Employer Statistics
1,-2.011290,EMP_F,N/A,Flag for Number of employees,Employer Statistics


In [15]:
api.geographies

,scheme,name,predicate_type,referenceDate,requires,optionalWithWCFor,wildcard
0,fips,us,NaN,NaN,NaN,NaN,NaN
1,fips,state,NaN,NaN,NaN,NaN,NaN
2,fips,county,NaN,NaN,[state],state,[state]


In [16]:
async def call_api(api: AsyncCensusDataEndpoint):
    return await api(['GEO_TTL', 'EMP'], {'county': '*'}, geo_in={'state': '36'})

In [17]:
task = loop.create_task(call_api(api))

In [18]:
task

<Task finished name='Task-33' coro=<call_api() done, defined at /tmp/ipykernel_497/2582133964.py:1> result=             ...s x 4 columns]>

In [18]:
task.result()

,GEO_TTL,EMP,state,county
0,"Allegany County, NY",10109,36,003
1,"Bronx County, NY",174205,36,005
2,"Warren County, NY",23157,36,113
3,"Cattaraugus County, NY",22098,36,009
4,"Cayuga County, NY",18224,36,011
...,...,...,...,...
58,"Westchester County, NY",353797,36,119
59,"Wyoming County, NY",7684,36,121
60,"Wayne County, NY",16702,36,117
61,"Yates County, NY",3693,36,123
